In [122]:
import torch.nn as nn
from torch.utils.data import DataLoader,Dataset
import numpy as np
import torch
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_digits
from sklearn.preprocessing import OneHotEncoder,StandardScaler
import torch.optim as optim
from tqdm import tqdm
from torchvision import datasets,transforms

In [123]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(0.1307,0.3081)
    ]
)

In [124]:
data_train = datasets.MNIST('data/train',train=True,transform=transform,download=True)
data_test = datasets.MNIST('data/test',train=False,transform=transform,download=True)

In [125]:
class MLP(nn.Module):
    def __init__(self, input_dims,output_dims):
        super(MLP,self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dims,32),
            nn.ReLU(),
            nn.Linear(32,16),
            nn.ReLU(),
            nn.Linear(16,output_dims)
        )
    def forward(self,x):
        return self.net(x)

In [126]:
model = MLP(784,10)

In [132]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(),lr=0.01,momentum=0.9)

In [133]:
train_loader = DataLoader(data_train,batch_size=256,shuffle=True)
test_loader = DataLoader(data_test,batch_size=256)

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = model.to(device)
num_epochs = 5

In [140]:


for epoch in range(num_epochs):
    model.train()
    train_pbar = tqdm(train_loader,position=0,leave=True)
    train_pbar.set_description(f'Epoch[{epoch+1}/{num_epochs}]')
    for x,y in train_pbar:
        optimizer.zero_grad()
        x = x.view(x.size(0),-1)
        x,y = x.to(device),y.to(device)
        y_pred = model.forward(x)
        loss = criterion(y_pred,y)
        loss.backward()
        optimizer.step()
        train_pbar.set_postfix({'train loss':loss.detach().item()})

Epoch[5/5]: 100%|██████████| 235/235 [00:15<00:00, 14.92it/s, train loss=0.0626]


In [141]:
correct = 0
total = 0
model.eval()

for x,y in test_loader:
    x = x.view(x.size(0),-1)
    x,y = x.to(device),y.to(device)
    with torch.no_grad():
        y_pred = model(x)
    _,idx1 = torch.max(y_pred,dim=1)
    correct += (idx1 == y).sum().item()
    total += len(y)
print("Acc:{}".format(correct/total))

Acc:0.9677
